In [21]:
import pandas as pd
import numpy as np
from numpy import arctan as atan, sqrt, sin, cos
from scipy.stats import pearsonr

### Augmented measures

The original sensor data $a_f$, $a_l$, $a_v$ (front, vertical, and lateral sensors, respectively) are augmented messures as described in Wickramasinghe et al (2017, appendix B). Firstly angles $\theta$, $\alpha$, and $\beta$ are defined as follows:


$\theta \approx \tan^{-1}(a_f / \sqrt{a_v^2 + a_l ^2})$

$\alpha \approx \tan^{-1}(a_l / a_v)$

$\beta \approx \tan^{-1}(a_l / a_f)$


Then approximate acceleration signals $a_x$, $a_y$, and $a_z$ are derived as shown below:

$a_x \approx a_v\sin(\theta) + a_f\cos(\theta)$

$a_y \approx a_v\sin(\alpha) + a_l\cos(\alpha)$

$a_z \approx 1 + a_v\cos(\theta)\cos(\alpha) + a_l\sin(\alpha) + a_f\sin(\theta)$

### Features

Features are taken over a window $X_{[t_i - \delta t, t_i]}$:

| Feature | Description |
|:-:|:-|
| $a_f$ | Frontal acceleration |
| $a_l$ | Lateral acceleration |
| $a_v$ | Vertical acceleration |
| $\theta$ | Angle on Sagittal plane |
| $\alpha$ | Angle on Coronal plane |
| $\beta$ | Angle on Transverse plane |
| $a_x$ | Anteroposterior acceleration |
| $a_y$ | Mediolateral acceleration |
| $a_z$ | Dorsvental acceleration |
| $\mbox{mean}$$(a_f)$; $\mbox{mean}$$(a_l)$; $\mbox{mean}$$(a_v)$ |
| $\mbox{max}$$(a_f)$; $\mbox{max}$$(a_l)$; $\mbox{max}$$(a_v)$ |
| $\mbox{min}$$(a_f)$; $\mbox{min}$$(a_l)$; $\mbox{min}$$(a_v)$ |
| $\mbox{corr}$$(a_f\mbox{,}a_l)$; $\mbox{corr}$$(a_f\mbox{,}a_v)$; $\mbox{corr}$$(a_l\mbox{,}a_v)$ |
| $\mbox{mean}$ $(a_x)$; $\mbox{mean}$ $(a_y)$; $\mbox{mean}$ $(a_z)$ | 
| $\mbox{mean}$$(\theta)$; $\mbox{mean}$$(\alpha)$; $\mbox{mean}$$(\beta)$ |
| $\mbox{max}$$(\theta)$; $\mbox{max}$$(\alpha)$; $\mbox{max}$$(\beta)$ |
| $\mbox{min}$$(\theta)$; $\mbox{min}$$(\alpha)$; $\mbox{min}$$(\beta)$ |

In [23]:
from pathlib import Path

import pandas as pd

DATA_ROOT = Path('..') / 'data' 

dfs = []
activity_labels = ['bed', 'chair', 'lying', 'ambulating']
default_names = ['time', 'front', 'vertical', 'lateral', 'sensor_id', 'rssi', 'phase', 'frequency', 'activity']
for data_file in Path(DATA_ROOT).rglob('d[12]p??[FM]'):
    df = pd.read_csv(data_file, names=default_names)
    df['activity_label'] = df['activity'].apply(lambda i: activity_labels[i - 1])
    df['gender_label'] = str(data_file)[-1]
    df['participant'] = data_file.name
    
    # Add a column indicating order of the activities for a particiapnt.
    df = df.sort_values(by=['time'])
    df['activity_sequence'] = (df['activity'].shift(1) != df['activity']).cumsum()
    dfs.append(df)

sensor_df = pd.concat(dfs, axis='index')
sensor_df = sensor_df.sort_values(by=['participant', 'time'])
sensor_df['room'] = sensor_df['participant'].str.slice(1, 2).astype(int)

sensor_df.head()

,time,front,vertical,lateral,sensor_id,rssi,phase,frequency,activity,activity_label,gender_label,participant,activity_sequence,room
0,0.00,0.27203,1.00820,-0.082102,1,-63.5,2.4252,924.25,1,bed,M,d1p01M,1,1
1,0.50,0.27203,1.00820,-0.082102,1,-63.0,4.7369,921.75,1,bed,M,d1p01M,1,1
2,1.50,0.44791,0.91636,-0.013684,1,-63.5,3.0311,923.75,1,bed,M,d1p01M,1,1
3,1.75,0.44791,0.91636,-0.013684,1,-63.0,2.0371,921.25,1,bed,M,d1p01M,1,1
4,2.50,0.34238,0.96229,-0.059296,1,-63.5,5.8920,920.25,1,bed,M,d1p01M,1,1


Each row $i$ of data in the dataframe is treated as the trailing edge of a frame of data from $t_i-\delta t$ to $t_i$, within each frame.

In [39]:
def corr(x, y):
    if len(x) == len(y) == 1:
        return 0
    return pearsonr(x, y)

def get_frame(df, t, delta_t):
    frame_df = df[(df['time'] > t - delta_t) & (df['time'] <= t)]
    
    if not len(frame_df):
        return None

    t = frame_df['time']
    af = frame_df['front']
    al = frame_df['lateral']
    av = frame_df['vertical']

    theta = atan(af / sqrt(av**2 + al**2))
    alpha = atan(al / av)
    beta = atan(al / af)

    ax = av * sin(theta) + af * cos(theta)
    ay = av * sin(alpha) + al * cos(alpha)
    az = 1 + av * cos(theta) * cos(alpha) + al * sin(alpha) + af * sin(theta)
    
    tmax_af = t[af.idxmax()]
    tmax_al = t[al.idxmax()]
    tmax_av = t[av.idxmax()]
    
    tmin_af = t[af.idxmin()]
    tmin_al = t[al.idxmin()]
    tmin_av = t[av.idxmin()]
    
    tmax_theta = t[theta.idxmax()]
    tmax_alpha = t[alpha.idxmax()]
    tmax_beta = t[beta.idxmax()]
    
    tmin_theta = t[theta.idxmin()]
    tmin_alpha = t[alpha.idxmin()]
    tmin_beta = t[beta.idxmin()]
    
    frame = dict({
        'class': frame_df['activity'][0],
        't': t.values[-1],
        'af': af.values[-1],
        'al': al.values[-1],
        'av': av.values[-1],
        'ar': np.sqrt(af.values[-1] ** 2 + al.values[-1] ** 2 + av.values[-1] ** 2),
        'ax': ax.values[-1],
        'ay': ay.values[-1],
        'az': az.values[-1],
        'af_mean': af.mean(),
        'al_mean': al.mean(),
        'av_mean': av.mean(),
        'af_max': af.max(),
        'al_max': al.max(),
        'av_max': av.max(),
        'af_min': af.min(),
        'al_min': al.min(),
        'av_min': av.min(),
        'coor_af-al': corr(af, al),
        'coor_af-av': corr(af, av),
        'coor_al-av': corr(al, av),
        'ax_mean': ax.mean(),
        'ay_mean': ay.mean(),
        'az_mean': az.mean(),
        'tmax_af<tmin_af': tmax_af < tmin_af,
        'tmax_al<tmin_al': tmax_al < tmin_al,
        'tmax_av<tmin_av': tmax_av < tmin_av,
        'theta': theta.values[-1],
        'alpha': alpha.values[-1],
        'beta': beta.values[-1],
        'theta_mean': theta.mean(),
        'alpha_mean': alpha.mean(),
        'beta_mean': beta.mean(),
        'theta_max': theta.max(),
        'alpha_max': alpha.max(),
        'beta_max': beta.max(),
        'theta_min': theta.min(),
        'alpha_min': alpha.min(),
        'beta_min': beta.min(),
        'tmax_theta<tmin_theta': tmax_theta < tmin_theta,
        'tmax_alpha<tmin_alpha': tmax_alpha < tmin_alpha,
        'tmax_beta<tmin_beta': tmax_beta < tmin_beta,
    })
    return pd.DataFrame(frame, index=frame_df.index[0:1])


def get_data(participant_df, t, delta_t):
    X = get_frame(participant_df, t, delta_t)
    y = X['class']
    X = X.drop('class', axis=1)
    return X, y


In [40]:
rooms = sensor_df['room'].unique()
room = rooms[0]

room_df = sensor_df[sensor_df['room'] == room]
participants = room_df['participant'].unique()
participant = participants[0]

participant_df = room_df[room_df['participant'] == participant]

In [41]:
delta_t = 2.0
t_freq = 0.5
min_t, max_t = participant_df['time'].min(), participant_df['time'].max()
t = min_t
while t <= max_t:
    frame_df = get_frame(participant_df, t, delta_t)
    t += t_freq
    break